In [2611]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
import re

import rouge
import difflib


In [2612]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Viktor_Shevchuk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Viktor_Shevchuk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2613]:
URL1 = "https://zakon.rada.gov.ua/laws/show/en/1207-18/conv/print"
URL2 = "https://zakon.rada.gov.ua/laws/show/en/1700-18/print"
page = requests.get(URL1)

# print(page.text) 


In [2614]:
soup = BeautifulSoup(page.content, "html.parser")
# soup

In [2615]:
paragraphs = soup.select('.rvts0 > .rvps2')
# paragraphs

In [2616]:
legal_stop_words = [
    "amended", "restated", "be stated", "supplemented", "section", "paragraph", "article", "as follows", "clauses" 
]


In [2617]:
def isSentenceWithStopWord(para):
    for word in legal_stop_words:
        sentenceWithStopWord = para.find(
            text=lambda t: word in t.text.lower()
        )

        if sentenceWithStopWord:
            return True


In [2618]:
def isProvisions(para):
   return para.find(
        text=lambda t: "final and transitional provisions" in t.text.lower()
    )

In [2619]:
def isArticleEnsuring(para):
  return para.find("span", class_="rvts37")


In [2620]:
def isArticleTitle(para):
    return para.find("span", class_="rvts9")


In [2621]:
def isClosingChar(letter):
  return letter == "." or letter == ")"


In [2622]:
def discardOrderedListNumbers(text):
    isFirstCharNumeric = text[0].isdigit()
    isSecondCharNumeric = text[1].isdigit()
    isThirdCharNumeric = text[2].isdigit()

    if isFirstCharNumeric and isClosingChar(text[1]):
        return text[2:].strip()
    elif isFirstCharNumeric and isSecondCharNumeric and isClosingChar(text[2]):
        return text[3:].strip()
    elif isFirstCharNumeric and isSecondCharNumeric and isThirdCharNumeric and isClosingChar(text[3]):
        return text[4:].strip()
    elif re.search(r"[a-z]", text[0]) and isClosingChar(text[1]):
        return text[2:].strip()

    return text


In [2623]:
def clean(paragraphs):
    cleaned = []
    articleEnsuring = False

    for para in paragraphs:
        if articleEnsuring and not isArticleTitle(para):
            continue
        elif articleEnsuring and isArticleTitle(para):
            articleEnsuring = False

        if isArticleEnsuring(para):
            articleEnsuring = True
            continue

        if isProvisions(para):
            return cleaned

        if not isSentenceWithStopWord(para) and not isArticleTitle(para):
            cleaned.append(discardOrderedListNumbers(para.text.strip()))

    return cleaned


In [2624]:
cleaned_paragraphs = nltk.sent_tokenize(
    ' '.join(clean(paragraphs)), language='english'
)

# cleaned_paragraphs

In [2625]:
featurizer = TfidfVectorizer(
    stop_words=stopwords.words('english'), 
    # norm='l1'
)
# featurizer


In [2626]:
X = featurizer.fit_transform(
    cleaned_paragraphs
)
# X

In [2627]:
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]

    return x.mean()

In [2628]:
scores = np.zeros(len(cleaned_paragraphs))
for i in range(len(cleaned_paragraphs)):
    score = get_sentence_score(X[i, :])
    scores[i] = score


In [2629]:
sort_idx = np.argsort(-scores)
# sort_idx

In [2630]:
for i in sort_idx[:10]:
    print("%.2f: %s" % (scores[i], cleaned_paragraphs[i]))

0.47: Re-transfer of the case shall not be allowed.
0.43: Under the Constitution of Ukraine, Ukraine is a sovereign and independent state.
0.38: The date of the beginning of the temporary occupation is 20 February 2014.
0.34: Relevant information on their search shall be provided to law enforcement agencies.
0.32: At the same time, information (note) about the place of stay of such a person shall not be entered in his/her passport.
0.32: Persons guilty of violating the requirements of this Law shall bear the responsibility provided for by law.
0.30: The modalities of economic activity in the temporarily occupied territory shall be determined by law.
0.30: The temporarily occupied territory of Ukraine (hereinafter referred to as the temporarily occupied territory) is an integral part of the territory of Ukraine, which is subject to the Constitution and laws of Ukraine.
0.28: Property rights within the temporarily occupied territory shall be protected under the legislation of Ukraine.
0.

In [2631]:
title = soup.find("span", class_="rvts23").text 
title


'On Ensuring Civil Rights and Freedoms, and the Legal Regime on the Temporarily Occupied Territory of Ukraine'

In [2632]:
def tfIDFSummarize(text):
    X = featurizer.fit_transform(text)
    scores = np.zeros(len(text))

    for i in range(len(text)):
        score = get_sentence_score(X[i, :])
        scores[i] = score

    sort_idx = np.argsort(-scores)
  
    summarized = ""

    for i in sort_idx[:10]:
        test = text[i]
        summarized += text[i] + " "

    return summarized.strip()


In [2633]:
tfIDFSummarize(cleaned_paragraphs)


'Re-transfer of the case shall not be allowed. Under the Constitution of Ukraine, Ukraine is a sovereign and independent state. The date of the beginning of the temporary occupation is 20 February 2014. Relevant information on their search shall be provided to law enforcement agencies. At the same time, information (note) about the place of stay of such a person shall not be entered in his/her passport. Persons guilty of violating the requirements of this Law shall bear the responsibility provided for by law. The modalities of economic activity in the temporarily occupied territory shall be determined by law. The temporarily occupied territory of Ukraine (hereinafter referred to as the temporarily occupied territory) is an integral part of the territory of Ukraine, which is subject to the Constitution and laws of Ukraine. Property rights within the temporarily occupied territory shall be protected under the legislation of Ukraine. The sovereignty of Ukraine shall apply to its entire te

In [2634]:
S = cosine_similarity(X)
# S

In [2635]:
S.shape

(78, 78)

In [2636]:
len(cleaned_paragraphs)


78

In [2637]:
# normalize similarity matrix
S /= S.sum(axis=1, keepdims=True)


In [2638]:
S[0].sum()

0.9999999999999999

In [2639]:
# uniform transition matrix
U = np.ones_like(S) / len(S)


In [2640]:
U[0].sum()

0.9999999999999996

In [2641]:
# smoothed similarity matrix
factor = 0.15 
S = (1 - factor) * S + factor * U

In [2642]:
S[0].sum()

0.9999999999999997

In [2643]:
# find the limiting / stationary distribution
eigenvals, eigenvecs = np.linalg.eig(S.T)


In [2644]:
eigenvals

array([1.        , 0.57096215, 0.482181  , 0.40338987, 0.36245362,
       0.34309105, 0.33124148, 0.32150353, 0.29937951, 0.28323814,
       0.27273049, 0.26984954, 0.25829957, 0.24513491, 0.2340806 ,
       0.22895872, 0.2251505 , 0.21043461, 0.20162603, 0.19213305,
       0.18609489, 0.18229309, 0.17677884, 0.17205491, 0.16674806,
       0.16072248, 0.15556234, 0.02657129, 0.15787016, 0.15402636,
       0.14963891, 0.0297785 , 0.03076921, 0.14517824, 0.13981344,
       0.1371983 , 0.03629587, 0.13045646, 0.1328084 , 0.12678317,
       0.12277042, 0.03962049, 0.04043359, 0.04212795, 0.11962235,
       0.04678052, 0.1155613 , 0.11363725, 0.04861277, 0.10803714,
       0.05021827, 0.10702008, 0.05087543, 0.0528662 , 0.10384827,
       0.10335395, 0.10009132, 0.09580748, 0.09073631, 0.09283672,
       0.05829744, 0.08765471, 0.08600608, 0.08328895, 0.06202772,
       0.06005818, 0.0987841 , 0.06292639, 0.05939792, 0.07961623,
       0.07870516, 0.065689  , 0.0695311 , 0.07686002, 0.07278

In [2645]:
eigenvecs[:, 0]


array([0.09798891, 0.08311021, 0.1126732 , 0.10512086, 0.1412555 ,
       0.05586681, 0.12408279, 0.14909163, 0.14076819, 0.10340315,
       0.16887935, 0.1011735 , 0.15538694, 0.12421994, 0.12734501,
       0.08198278, 0.11254355, 0.11595152, 0.0825056 , 0.11805816,
       0.13252062, 0.11123564, 0.0832884 , 0.06237141, 0.08003133,
       0.11797849, 0.12961233, 0.11497828, 0.13200868, 0.10417595,
       0.12785987, 0.10178179, 0.13961774, 0.08223921, 0.08459816,
       0.096545  , 0.08191448, 0.13754674, 0.10496958, 0.13765053,
       0.10328712, 0.10106222, 0.13795963, 0.14162163, 0.12957263,
       0.11749498, 0.12853689, 0.12136265, 0.1043849 , 0.11141806,
       0.11190817, 0.10093836, 0.12309887, 0.12776457, 0.07378612,
       0.06391599, 0.11070335, 0.10079633, 0.08250008, 0.09866223,
       0.06960368, 0.11074409, 0.07178706, 0.07856162, 0.10429558,
       0.09705767, 0.10179147, 0.08210656, 0.10016309, 0.09630253,
       0.11803207, 0.15841944, 0.13143195, 0.149329  , 0.13925

In [2646]:
eigenvecs[:, 0].dot(S)


array([0.09798891, 0.08311021, 0.1126732 , 0.10512086, 0.1412555 ,
       0.05586681, 0.12408279, 0.14909163, 0.14076819, 0.10340315,
       0.16887935, 0.1011735 , 0.15538694, 0.12421994, 0.12734501,
       0.08198278, 0.11254355, 0.11595152, 0.0825056 , 0.11805816,
       0.13252062, 0.11123564, 0.0832884 , 0.06237141, 0.08003133,
       0.11797849, 0.12961233, 0.11497828, 0.13200868, 0.10417595,
       0.12785987, 0.10178179, 0.13961774, 0.08223921, 0.08459816,
       0.096545  , 0.08191448, 0.13754674, 0.10496958, 0.13765053,
       0.10328712, 0.10106222, 0.13795963, 0.14162163, 0.12957263,
       0.11749498, 0.12853689, 0.12136265, 0.1043849 , 0.11141806,
       0.11190817, 0.10093836, 0.12309887, 0.12776457, 0.07378612,
       0.06391599, 0.11070335, 0.10079633, 0.08250008, 0.09866223,
       0.06960368, 0.11074409, 0.07178706, 0.07856162, 0.10429558,
       0.09705767, 0.10179147, 0.08210656, 0.10016309, 0.09630253,
       0.11803207, 0.15841944, 0.13143195, 0.149329  , 0.13925

In [2647]:
scores = eigenvecs[:, 0] / eigenvecs[:, 0].sum()
sort_idx = np.argsort(-scores)


In [2648]:
for i in sort_idx[:10]:
    print("%.2f: %s" % (scores[i], cleaned_paragraphs[i]))


0.02: Ukraine shall take all necessary measures to guarantee human and civil rights and freedoms provided for by the Constitution and laws of Ukraine, international treaties, to all citizens of Ukraine residing within the temporarily occupied territory.
0.02: In case of violation of the provisions of this Law, state bodies of Ukraine shall apply the mechanisms provided for by the laws of Ukraine and the norms of international law in order to protect the peace, security, rights, freedoms and legitimate interests of citizens of Ukraine located within the temporarily occupied territory, as well as the legitimate interests of the state of Ukraine.
0.02: Responsibility for violating human and civil rights and freedoms defined by the Constitution and laws of Ukraine within the temporarily occupied territory shall be assigned to the Russian Federation as an occupying state under the norms and principles of international law.
0.02: If it is impossible to carry out activities within the tempora

In [2649]:
def textRankSummarize(text, factor=0.15):
    X = featurizer.fit_transform(text)
    S = cosine_similarity(X)  # compute similarity matrix
    S /= S.sum(axis=1, keepdims=True)  # normalize similarity matrix
    U = np.ones_like(S) / len(S)  # uniform transition matrix
    S = (1 - factor) * S + factor * U  # smooth similarity matrix
    # find the limiting / stationary distribution
    eigenvals, eigenvecs = np.linalg.eig(S.T)
    scores = eigenvecs[:, 0] / eigenvecs[:, 0].sum()  # compute scores
    sort_idx = np.argsort(-scores)  # sort scores

    summarized = ""

    for i in sort_idx[:10]:
        test = text[i]
        summarized += text[i] + " "

    return summarized.strip()


In [2650]:
textRankSummarize(cleaned_paragraphs)


'Ukraine shall take all necessary measures to guarantee human and civil rights and freedoms provided for by the Constitution and laws of Ukraine, international treaties, to all citizens of Ukraine residing within the temporarily occupied territory. In case of violation of the provisions of this Law, state bodies of Ukraine shall apply the mechanisms provided for by the laws of Ukraine and the norms of international law in order to protect the peace, security, rights, freedoms and legitimate interests of citizens of Ukraine located within the temporarily occupied territory, as well as the legitimate interests of the state of Ukraine. Responsibility for violating human and civil rights and freedoms defined by the Constitution and laws of Ukraine within the temporarily occupied territory shall be assigned to the Russian Federation as an occupying state under the norms and principles of international law. If it is impossible to carry out activities within the temporarily occupied territory

Libraries for text summarization 

In [2651]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer


In [2652]:
parser = PlaintextParser.from_string(cleaned_paragraphs, Tokenizer("english"))
parser 

In [2653]:
summarizer = LsaSummarizer()
summarizer


In [2654]:
summary = summarizer(parser.document, sentences_count=10)
summary


(<Sentence: ', 'Elderly citizens, persons with disabilities, children with disabilities and other citizens of Ukraine experiencing difficult life circumstances and residing within the temporarily occupied territory shall have the right to be provided with social services under the legislation of Ukraine.>,
 <Sentence: ', 'Foreigners and stateless persons permanently residing in Ukraine, persons who have been granted refugee status in Ukraine, persons in need of additional or temporary protection, and persons who have been granted the status of a foreign Ukrainian, who are legally staying in Ukraine and live within the temporarily occupied territory, shall have the right to get or continue getting a certain qualification within the territory of other regions of Ukraine following the procedure defined by the laws of Ukraine “On Higher Education” and “On Professional Pre-higher Education".>,
 <Sentence: ', 'During presidential, parliamentary elections and all-Ukrainian referendums, voting

TextRank vs Seq2Seq vs BART

The hardest NLP tasks are the ones where the output isn’t a single label or value (like Classification and Regression), but a full new text (like Translation, Summarization and Conversation).

Text summarization is the problem of reducing the number of sentences and words of a document without changing its meaning. There are different techniques to extract information from raw text data and use it for a summarization model, overall they can be categorized as Extractive and Abstractive. Extractive methods select the most important sentences within a text (without necessarily understanding the meaning), therefore the result summary is just a subset of the full text. On the contrary, Abstractive models understand the semantics of the text and generate a meaningful summary. 

I’ll keep On Ensuring Civil Rights and Freedoms, and the Legal Regime on the Temporarily Occupied Territory of Ukraine summary on dentons to compare different models.

In [2655]:
humanSummary="""
On 15 April 2014 the Ukrainian Parliament passed the Law of Ukraine “On Ensuring Civil Rights and Freedoms and the Legal Regime on Temporarily Occupied Territory of Ukraine” (the "Law").
Specifically, the Law determines a legal regime of occupied territories as well as sets forth special procedures for the operation of governmental authorities, bodies of local self-government, enterprises and organizations, under such legal regime, provides for protection of human and civil rights and freedoms as well as rights, freedoms and lawful interests of legal entities and indicates Ukrainian courts that will consider disputes arising in Crimea during the occupation.
It defines "occupied territory" as including the land territory of the Autonomous Republic of Crimea and the city of Sevastopol as well as their domestic waters; domestic waters and territorial sea of Ukraine adjacent to the coast of the Crimean peninsula; the territory of an adjacent area, exclusive economic zone and continental shelf along the coast of the Crimean peninsula subject to jurisdiction of Ukrainian bodies of state power in accordance with provisions of international law, the Constitution and laws of Ukraine; underwater space within the territorial sea; air space above these territories.
For as long as Crimea remains occupied, the "occupied territory" will be subjected to a special legal regime with respect to the border crossing with the "occupied territory", carrying out business activity, entering into transactions, holding elections and referenda, exercise of other human and civil rights and freedoms as well as a special procedure for ensuring human and civil rights and freedoms of Ukrainian citizens residing in the temporarily occupied territories of Ukraine.
1) Activities of state and local self-governance bodies created in the occupied territories in breach of Ukrainian law shall be prohibited, and their regulations and acts shall be illegal and will not be recognized in Ukraine - e.g. permits, authorizations, allocation of  land plots.
2) The Law specifically deals with transfers of Crimea-based immovable property by setting forth that ownership rights to immovable property may be transferred and acquired in accordance with Ukrainian law. As a result, Crimea-based immovable property may be transferred outside Crimea and notarized by any Ukrainian notary.
3) Disputes arising in Crimea during the occupation will be considered by respective civil, commercial and administrative courts in the city of Kyiv.
4) If provisions of the Law are violated, Ukrainian authorities shall apply mechanisms provided for by Ukrainian law and provisions of international laws both in Ukrainian and international courts.
Business activities in Crimea will be covered by a separate law intended to specifically address the "doing business in Crimea" aspect.
Generally, the Law limits business activity in the territory of Crimea and, obviously, persons engaged in business activity in Crimea may be prosecuted by Ukraine and the international community (i.e., the countries that did not recognize the occupation of Crimea by the Russian Federation). Considering this, the Ministry of Justice of Ukraine introduced a simplified procedure for Crimea-registered businesses to change their registered addresses. Accordingly, Crimea-registered businesses may now register anywhere in Ukraine and continue with their businesses.
"""

How can we evaluate the results? 

In [2656]:
'''
Calculate ROUGE score.
:parameter    
    :param y_test: string or list    
    :param predicted: string or list
'''
def evaluate_summary(y_test, predicted):
    rouge_score = rouge.Rouge()
    scores = rouge_score.get_scores(y_test, predicted, avg=True)
    score_1 = round(scores['rouge-1']['f'], 2)
    score_2 = round(scores['rouge-2']['f'], 2)
    score_L = round(scores['rouge-l']['f'], 2)
    print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:",
          score_2, "--> avg rouge:", round(np.mean(
              [score_1, score_2, score_L]), 2))


In [2657]:
evaluate_summary(
    humanSummary,
    tfIDFSummarize(cleaned_paragraphs)
)


rouge1: 0.22 | rouge2: 0.07 | rougeL: 0.07 --> avg rouge: 0.16


The results show that 22% of unigrams (ROUGE-1) and 7% of bigrams (ROUGE-2) are present in both summaries, while the longest common subsequences (ROUGE-L) match by 7%. Overall, the average score is 16%. ROUGE scores don’t measure how fluent the summary is. For that experts recomend using the good old human eye. 

In [2658]:
evaluate_summary(
    humanSummary,
    textRankSummarize(cleaned_paragraphs)
)


rouge1: 0.42 | rouge2: 0.23 | rougeL: 0.23 --> avg rouge: 0.35


Visualization: I will display 2 texts, i.e. the summary and the original text, or the predicted summary and the real summary, and highlight the matching parts. It highlights the matching substrings of two texts. It can be used on word-level.

In [2659]:
'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''


def utils_split_sentences(a, b):
    # find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks()
                 if block.size > 20]

    # difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)

    # work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        # a
        string = a[0: first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a: m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size: next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size:]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        # b
        string = b[0: first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b: m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size: next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size:]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]

    return lst_a, lst_b

'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''


def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()

    # highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append(
                '<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)

    # highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append(
                    '<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b)
    second_text = ' '.join(second_text)

    # concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text + '<br><br>' + second_text
    return final_text


In [2660]:
match = display_string_matching(
    humanSummary,
    tfIDFSummarize(cleaned_paragraphs),
    both=True,
    sentences=False,
    titles=["Real Summary", "Predicted Summary"]
)


In [2661]:
from IPython.core.display import display, HTML
display(HTML(match))


C:\Users\Viktor_Shevchuk\AppData\Local\Temp\ipykernel_17864\3609584809.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2662]:
match = display_string_matching(
    humanSummary,
    textRankSummarize(cleaned_paragraphs),
    both=True,
    sentences=False,
    titles=["Real Summary", "Predicted Summary"]
)


In [2663]:
from IPython.core.display import display, HTML
display(HTML(match))


C:\Users\Viktor_Shevchuk\AppData\Local\Temp\ipykernel_17864\3609584809.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2664]:
match = display_string_matching(
    ' '.join(cleaned_paragraphs),
    tfIDFSummarize(cleaned_paragraphs), 
    both=True,
    titles=["Full Text", "Predicted Summary"]
)

display(HTML(match))


In [2665]:
match = display_string_matching(
    ' '.join(cleaned_paragraphs),
    textRankSummarize(cleaned_paragraphs),
    both=True,
    titles=["Full Text", "Predicted Summary"]
)

display(HTML(match))


The prediction has most of the information mentioned in the original summary. As expected from an Extractive algorithms, the predicted summary is fully contained in the text: the model considers those the most important sentences.